In [ ]:
pip install yfinance

In [1]:
from helpers import *
import yfinance as yf
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.stats import norm
from datetime import date, timedelta, datetime

/Users/irfanjamil/anaconda3/lib/python3.10/site-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


In [2]:
from options_helpers import *
from scipy.ndimage import gaussian_filter1d
from scipy.interpolate import CubicSpline

In [3]:
annualized_rate = .05374
ir = np.log(1+annualized_rate)

In [16]:
'''
gets hourly SPX and NDX options data on an hourly basis from some specified date onwards from Polygon and stores data in df
'''
import requests
params_ = {'adjusted': 'true', 'sort': 'asc', 'apiKey' : 'vGQU0FoV2y68f1aA4hFlncpgRNiJVroO'}
d = {'underlying': [], 'strike': [], 'type': [], 'date': [], 'ts': [], 'price': []}

ndx_df, spx_df = get_ndx_spx_data()
    
ndx_df = ndx_df[(ndx_df['Date'] >= date(2024, 3, 18)) & (ndx_df['Date'] <= date(2024, 3, 18))][['Date', 'Open', 'Close']].reset_index(drop=True)
spx_df = spx_df[(spx_df['Date'] >= date(2024, 3, 18)) & (spx_df['Date']<= date(2024, 3, 18))][['Date', 'Open', 'Close']].reset_index(drop=True) #get daily index data for 2024 onwards

exception_rate = []

# -- NDX below --

for idx, row in ndx_df.iterrows():
    cur_dt = date.today()
    year = str(cur_dt.year - 2000)
    month = cur_dt.month
    if month < 10:
        month = '0' + str(month)
    else:
        month = str(month)
    day = cur_dt.day
    if day < 10:
        day = '0' + str(day)
    else:
        day = str(day)
        
    close_ = row['Open']
    #get OTM calls and puts data
    lb_call_strike = close_ + 10 - close_ % 10
    ub_put_strike = close_ - close_ % 10
    call_strikes = np.arange(lb_call_strike, lb_call_strike + 500, 10)
    put_strikes = np.arange(ub_put_strike - 500, ub_put_strike + 10, 10)
    strikes = list(call_strikes) + list(put_strikes)
    day_data_ret_err = 0
    for sp in strikes:

        put_ticker = 'NDXP' + year + month + day + 'P' +  str(int(sp)) + '000'
        call_ticker = 'NDXP'+ year + month + day + 'C' + str(int(sp)) + '000'
        put_url = 'https://api.polygon.io/v2/aggs/ticker/O:{}/range/{}/{}/{}/{}'.format(put_ticker, '1', 'hour', cur_dt.isoformat(), cur_dt.isoformat())
        call_url = 'https://api.polygon.io/v2/aggs/ticker/O:{}/range/{}/{}/{}/{}'.format(call_ticker, '1', 'hour', cur_dt.isoformat(), cur_dt.isoformat())
        
        r = requests.get(put_url, params=params_).json()
        try:
            for res in r['results']:
                option_close = res['o']
                ts = res['t']/1000
                d['strike'].append(int(sp))
                d['underlying'].append('NDX')
                d['type'].append('put')
                d['date'].append(cur_dt)
                d['ts'].append(datetime.fromtimestamp(ts))
                d['price'].append(option_close)
        except:
            day_data_ret_err += 1
            
        r = requests.get(call_url, params=params_).json()
        try:
            for res in r['results']:
                option_close = res['o']
                ts = res['t']/1000
                d['strike'].append(int(sp))
                d['underlying'].append('NDX')
                d['type'].append('call')
                d['date'].append(cur_dt)
                d['ts'].append(datetime.fromtimestamp(ts))
                d['price'].append(option_close)
        except:
            day_data_ret_err += 1
        
        
    if day_data_ret_err > 0:
        exception_rate.append(day_data_ret_err/(2*len(strikes)))

# -- SPX below --

for idx, row in spx_df.iterrows():
    cur_dt = date.today()
    year = str(cur_dt.year - 2000)
    month = cur_dt.month
    if month < 10:
        month = '0' + str(month)
    else:
        month = str(month)
    day = cur_dt.day
    if day < 10:
        day = '0' + str(day)
    else:
        day = str(day)
        
    close_ = row['Open']
    #get OTM calls and puts data
    lb_call_strike = close_ + 5 - close_ % 5
    ub_put_strike = close_ - close_ % 5
    call_strikes = np.arange(lb_call_strike, lb_call_strike + 155, 5)
    put_strikes = np.arange(ub_put_strike - 150, ub_put_strike + 5, 5)
    strikes = list(call_strikes) + list(put_strikes)
    day_data_ret_err = 0
    for sp in strikes:

        put_ticker = 'SPXW' + year + month + day + 'P' + '0' + str(int(sp)) + '000'
        call_ticker = 'SPXW'+ year + month + day + 'C' + '0' + str(int(sp)) + '000'
        put_url = 'https://api.polygon.io/v2/aggs/ticker/O:{}/range/{}/{}/{}/{}'.format(put_ticker, '1', 'hour', cur_dt.isoformat(), cur_dt.isoformat())
        call_url = 'https://api.polygon.io/v2/aggs/ticker/O:{}/range/{}/{}/{}/{}'.format(call_ticker, '1', 'hour', cur_dt.isoformat(), cur_dt.isoformat())
        
        r = requests.get(put_url, params=params_).json()
        try:
            for res in r['results']:
                option_close = res['o']
                ts = res['t']/1000
                d['strike'].append(int(sp))
                d['underlying'].append('SPX')
                d['type'].append('put')
                d['date'].append(cur_dt)
                d['ts'].append(datetime.fromtimestamp(ts))
                d['price'].append(option_close)
        except:
            day_data_ret_err += 1
            
        r = requests.get(call_url, params=params_).json()
        try:
            for res in r['results']:
                option_close = res['o']
                ts = res['t']/1000
                d['strike'].append(int(sp))
                d['underlying'].append('SPX')
                d['type'].append('call')
                d['date'].append(cur_dt)
                d['ts'].append(datetime.fromtimestamp(ts))
                d['price'].append(option_close)
        except:
            day_data_ret_err += 1
        
        
    if day_data_ret_err > 0:
        exception_rate.append(day_data_ret_err/(2*len(strikes)))

print(sum(exception_rate)/len(exception_rate))
df = pd.DataFrame(data=d)

ndx_opt_data = df[df['underlying'] == 'NDX']
spx_opt_data = df[df['underlying'] == 'SPX']
ndx_df['date'] = ndx_df.apply(lambda x: x['Date'], axis=1)
spx_df['date'] = spx_df.apply(lambda x: x['Date'], axis=1)

ndx_df = ndx_df[['date', 'Open', 'Close']]
spx_df = spx_df[['date', 'Open', 'Close']]

ndx_opt_data = ndx_opt_data.merge(ndx_df, how='inner', on='date')

ndx_itm_data = ndx_opt_data[((ndx_opt_data['strike'] > ndx_opt_data['Open']) & (ndx_opt_data['type']=='put')) | ((ndx_opt_data['strike'] < ndx_opt_data['Open']) & (ndx_opt_data['type']=='call'))]
ndx_otm_data = ndx_opt_data[((ndx_opt_data['strike'] > ndx_opt_data['Open']) & (ndx_opt_data['type']=='call')) | ((ndx_opt_data['strike'] < ndx_opt_data['Open']) & (ndx_opt_data['type']=='put'))]

spx_opt_data = spx_opt_data.merge(spx_df, how='inner', on='date')
spx_itm_data = spx_opt_data[((spx_opt_data['strike'] > spx_opt_data['Open']) & (spx_opt_data['type']=='put')) | ((spx_opt_data['strike'] < spx_opt_data['Open']) & (spx_opt_data['type']=='call'))]
spx_otm_data = spx_opt_data[((spx_opt_data['strike'] > spx_opt_data['Open']) & (spx_opt_data['type']=='call')) | ((spx_opt_data['strike'] < spx_opt_data['Open']) & (spx_opt_data['type']=='put'))]


df_itm = pd.concat([ndx_itm_data, spx_itm_data])
df_otm = pd.concat([ndx_otm_data, spx_otm_data])
df_itm['hour'] = df_itm.apply(lambda x: x['ts'].hour, axis=1)
df_otm['hour'] = df_otm.apply(lambda x: x['ts'].hour, axis=1)

df_itm = df_itm[df_itm['hour']==9]
df_otm = df_otm[df_otm['hour']==9]

df_itm = df_itm.sort_values(by=['date', 'strike']).reset_index(drop=True)
df_otm = df_otm.sort_values(by=['date', 'strike']).reset_index(drop=True)


0.13557968700095815


In [17]:
df_otm[(df_otm['underlying']=='SPX') & (df_otm['strike'] > 5100) & (df_otm['strike'] < 5200)]

,underlying,strike,type,date,ts,price,Open,Close,hour
21,SPX,5105,put,2024-03-18,2024-03-18 09:00:00,0.40,5154.77002,5149.419922,9
22,SPX,5110,put,2024-03-18,2024-03-18 09:00:00,0.50,5154.77002,5149.419922,9
23,SPX,5115,put,2024-03-18,2024-03-18 09:00:00,0.65,5154.77002,5149.419922,9
24,SPX,5120,put,2024-03-18,2024-03-18 09:00:00,0.85,5154.77002,5149.419922,9
25,SPX,5125,put,2024-03-18,2024-03-18 09:00:00,1.10,5154.77002,5149.419922,9
26,SPX,5130,put,2024-03-18,2024-03-18 09:00:00,1.55,5154.77002,5149.419922,9
27,SPX,5135,put,2024-03-18,2024-03-18 09:00:00,2.15,5154.77002,5149.419922,9
28,SPX,5140,put,2024-03-18,2024-03-18 09:00:00,2.80,5154.77002,5149.419922,9
29,SPX,5145,put,2024-03-18,2024-03-18 09:00:00,4.10,5154.77002,5149.419922,9
30,SPX,5150,put,2024-03-18,2024-03-18 09:00:00,5.10,5154.77002,5149.419922,9


In [18]:
df_otm[(df_otm['underlying']=='NDX') & (df_otm['strike'] > 17950) & (df_otm['strike'] < 18100)]

,underlying,strike,type,date,ts,price,Open,Close,hour
102,NDX,17960,put,2024-03-18,2024-03-18 09:00:00,23.30,18024.199219,17985.007812,9
103,NDX,17970,put,2024-03-18,2024-03-18 09:00:00,19.27,18024.199219,17985.007812,9
104,NDX,17980,put,2024-03-18,2024-03-18 09:00:00,27.50,18024.199219,17985.007812,9
105,NDX,17990,put,2024-03-18,2024-03-18 09:00:00,20.10,18024.199219,17985.007812,9
106,NDX,18000,put,2024-03-18,2024-03-18 09:00:00,31.80,18024.199219,17985.007812,9
107,NDX,18010,put,2024-03-18,2024-03-18 09:00:00,39.30,18024.199219,17985.007812,9
108,NDX,18020,put,2024-03-18,2024-03-18 09:00:00,45.80,18024.199219,17985.007812,9
109,NDX,18030,call,2024-03-18,2024-03-18 09:00:00,83.20,18024.199219,17985.007812,9
110,NDX,18040,call,2024-03-18,2024-03-18 09:00:00,58.50,18024.199219,17985.007812,9
111,NDX,18050,call,2024-03-18,2024-03-18 09:00:00,45.08,18024.199219,17985.007812,9


In [19]:
df_otm['est_iv'] = df_otm.apply(lambda x: implied_volatility_call(x['price'], x['Open'], x['strike'], 1/252, ir) if x['type']=='call' else implied_volatility_put(x['price'], x['Open'], x['strike'], 1/252, ir) , axis=1)


found on 6th iteration
difference is equal to 2.120851963355319e-07
found on 6th iteration
difference is equal to 4.654213537208518e-07
found on 6th iteration
difference is equal to 9.469932756162036e-07
found on 6th iteration
difference is equal to 1.8001207855861123e-06
found on 6th iteration
difference is equal to 3.2179184870501087e-06
found on 6th iteration
difference is equal to 5.440797637132344e-06
found on 6th iteration
difference is equal to 8.744534945745452e-06
found on 6th iteration
difference is equal to 1.9512410318056617e-07
found on 6th iteration
difference is equal to 3.557328099590418e-07
found on 6th iteration
difference is equal to 2.789335404144999e-05
found on 6th iteration
difference is equal to 9.86967437194286e-07
found on 6th iteration
difference is equal to 1.5143329662181682e-06
found on 6th iteration
difference is equal to 2.2090575711730676e-06
found on 6th iteration
difference is equal to 7.886787146631213e-05
found on 6th iteration
difference is equal t

found on 4th iteration
difference is equal to 8.32128712602298e-08
found on 4th iteration
difference is equal to 6.581859448573368e-07
found on 4th iteration
difference is equal to 1.2170641721098718e-05
found on 4th iteration
difference is equal to 2.774133755334418e-05
found on 5th iteration
difference is equal to 5.802300862001175e-10
found on 5th iteration
difference is equal to 3.936384107561253e-09
found on 5th iteration
difference is equal to 3.497920264550203e-08
found on 5th iteration
difference is equal to 8.111503557728383e-08
found on 5th iteration
difference is equal to 1.037028964478992e-06
found on 5th iteration
difference is equal to 2.54161412160947e-06
found on 5th iteration
difference is equal to 1.0353160813103557e-06
found on 5th iteration
difference is equal to 2.3851011383690945e-06
found on 6th iteration
difference is equal to 1.5122411656243173e-08
found on 5th iteration
difference is equal to 3.284357038180197e-05
found on 6th iteration
difference is equal to 

c = S_0 N(d1) - K * e^(-rT) * N(d2)

c = e^ (-rT) * E [max(S-K, 0)]
 = e ^ (-rT) * sum (p(Z) * (Z-k)) for Z > k
 = e ^ (-rT) * p (S > k) * E[S-K| S > K]
 = e ^ (-rT) * p(S > K) * (E[S| S > K] - K)
 = e ^ (-rT) * p(S > K) * (E[S| S > K]) - K * e ^ (-rT) * p(S > K)
 
 so p(S>K) ~=~ N(d2)
 where d2 = (ln(S_0/K) + (r-(sigma**2)/2) * T)/(sigma * sqrt(T))
 

In [20]:
def calc_geq_strike_prob(S_0, K, r, sigma, T):
    d2 = (np.log(S_0/K) + (r - (sigma**2)/2)*T)/(sigma * np.sqrt(T))
    return norm.cdf(d2)

In [21]:
df_otm['geq_strike_prob'] = df_otm.apply(lambda x:calc_geq_strike_prob(x['Open'], x['strike'], ir, x['est_iv'], 1/252), axis=1)

In [25]:
df_otm[(df_otm['underlying']=='SPX') & (df_otm['strike'] > 5000) & (df_otm['strike'] < 5250)]

,underlying,strike,type,date,ts,price,Open,Close,hour,est_iv,geq_strike_prob
1,SPX,5005,put,2024-03-18,2024-03-18 09:00:00,0.05,5154.77002,5149.419922,9,0.171639,0.996936
2,SPX,5010,put,2024-03-18,2024-03-18 09:00:00,0.05,5154.77002,5149.419922,9,0.166424,0.996850
3,SPX,5015,put,2024-03-18,2024-03-18 09:00:00,0.05,5154.77002,5149.419922,9,0.161199,0.996760
4,SPX,5020,put,2024-03-18,2024-03-18 09:00:00,0.05,5154.77002,5149.419922,9,0.155965,0.996663
5,SPX,5025,put,2024-03-18,2024-03-18 09:00:00,0.05,5154.77002,5149.419922,9,0.150720,0.996560
6,SPX,5030,put,2024-03-18,2024-03-18 09:00:00,0.05,5154.77002,5149.419922,9,0.145465,0.996449
7,SPX,5035,put,2024-03-18,2024-03-18 09:00:00,0.10,5154.77002,5149.419922,9,0.151150,0.993542
8,SPX,5040,put,2024-03-18,2024-03-18 09:00:00,0.10,5154.77002,5149.419922,9,0.145532,0.993322
9,SPX,5045,put,2024-03-18,2024-03-18 09:00:00,0.05,5154.77002,5149.419922,9,0.129623,0.996063
10,SPX,5050,put,2024-03-18,2024-03-18 09:00:00,0.10,5154.77002,5149.419922,9,0.134245,0.992828


In [24]:
df_otm[(df_otm['underlying']=='NDX') & (df_otm['strike'] > 17900) & (df_otm['strike'] < 18100)]

,underlying,strike,type,date,ts,price,Open,Close,hour,est_iv,geq_strike_prob
97,NDX,17910,put,2024-03-18,2024-03-18 09:00:00,13.95,18024.199219,17985.007812,9,0.118379,0.809612
98,NDX,17920,put,2024-03-18,2024-03-18 09:00:00,16.94,18024.199219,17985.007812,9,0.121364,0.782807
99,NDX,17930,put,2024-03-18,2024-03-18 09:00:00,20.00,18024.199219,17985.007812,9,0.123578,0.756756
100,NDX,17940,put,2024-03-18,2024-03-18 09:00:00,13.30,18024.199219,17985.007812,9,0.097305,0.786612
101,NDX,17950,put,2024-03-18,2024-03-18 09:00:00,20.80,18024.199219,17985.007812,9,0.111691,0.729833
102,NDX,17960,put,2024-03-18,2024-03-18 09:00:00,23.30,18024.199219,17985.007812,9,0.110836,0.704474
103,NDX,17970,put,2024-03-18,2024-03-18 09:00:00,19.27,18024.199219,17985.007812,9,0.092956,0.707757
104,NDX,17980,put,2024-03-18,2024-03-18 09:00:00,27.50,18024.199219,17985.007812,9,0.105409,0.654581
105,NDX,17990,put,2024-03-18,2024-03-18 09:00:00,20.10,18024.199219,17985.007812,9,0.079398,0.662301
106,NDX,18000,put,2024-03-18,2024-03-18 09:00:00,31.80,18024.199219,17985.007812,9,0.098045,0.597960


In [ ]:
df_otm[(df_otm['underlying']=='NDX') & (df_otm['strike'] > 17900) & (df_otm['strike'] < 18100)]